![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_XlmRoBertaForQuestionAnswering.ipynb)

## Import ONNX XlmRoBertaForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `XlmRoBertaForQuestionAnswering` is only available since in `Spark NLP 5.2.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import XLM-RoBERTa models trained/fine-tuned for question answering via `XlmRoBertaForQuestionAnswering` or `TFXlmRoBertaForQuestionAnswering`. These models are usually under `Question Answering` category and have `xlm-roberta` in their labels
- Reference: [TFXlmRoBertaForQuestionAnswering](https://huggingface.co/docs/transformers/model_doc/xlm-roberta#transformers.TFXLMRobertaForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=xlm-roberta&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [1]:
!pip install -q --upgrade transformers[onnx]==4.52.3 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use ['deepset/xlm-roberta-base-squad2'](https://huggingface.co/'deepset/xlm-roberta-base-squad2') model from HuggingFace as an example as an example and load it as a `ORTModelForQuestionAnswering`, representing an ONNX model.
- In addition to the XLM-RoBERTa model, we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import XLMRobertaTokenizer
from optimum.onnxruntime import ORTModelForQuestionAnswering

MODEL_NAME = 'deepset/xlm-roberta-base-squad2'
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

('onnx_models/deepset/xlm-roberta-base-squad2/tokenizer_config.json',
 'onnx_models/deepset/xlm-roberta-base-squad2/special_tokens_map.json',
 'onnx_models/deepset/xlm-roberta-base-squad2/sentencepiece.bpe.model',
 'onnx_models/deepset/xlm-roberta-base-squad2/added_tokens.json')

Let's have a look inside this directory and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 1089088
-rw-r--r-- 1 root root        733 Jun 17 04:47 config.json
-rw-r--r-- 1 root root 1110137819 Jun 17 04:47 model.onnx
-rw-r--r-- 1 root root    5069051 Jun 17 04:47 sentencepiece.bpe.model
-rw-r--r-- 1 root root        280 Jun 17 04:47 special_tokens_map.json
-rw-r--r-- 1 root root       1260 Jun 17 04:47 tokenizer_config.json


- As you can see, we need to move `sentencepiece.bpe.model` from the tokenizer to `assets` folder which Spark NLP will look for

In [4]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/sentencepiece.bpe.model {EXPORT_PATH}/assets

In [5]:
!ls -lR {EXPORT_PATH}

onnx_models/deepset/xlm-roberta-base-squad2:
total 1084140
drwxr-xr-x 2 root root       4096 Jun 17 04:47 assets
-rw-r--r-- 1 root root        733 Jun 17 04:47 config.json
-rw-r--r-- 1 root root 1110137819 Jun 17 04:47 model.onnx
-rw-r--r-- 1 root root        280 Jun 17 04:47 special_tokens_map.json
-rw-r--r-- 1 root root       1260 Jun 17 04:47 tokenizer_config.json

onnx_models/deepset/xlm-roberta-base-squad2/assets:
total 4952
-rw-r--r-- 1 root root 5069051 Jun 17 04:47 sentencepiece.bpe.model


Voila! We have our `sentencepiece.bpe.model` inside assets directory

## Import and Save XlmRoBertaForQuestionAnswering in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 41.0 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `RoBertaForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [8]:
from sparknlp.annotator import XlmRoBertaForQuestionAnswering

spanClassifier = XlmRoBertaForQuestionAnswering.loadSavedModel(
     EXPORT_PATH,
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(EXPORT_PATH))

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your XlmRoBertaForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {EXPORT_PATH}_spark_nlp_onnx

total 1089248
drwxr-xr-x 2 root root       4096 Jun 17 04:52 metadata
-rw-r--r-- 1 root root 1110307348 Jun 17 04:53 xlm_roberta_classification_onnx
-rw-r--r-- 1 root root    5069051 Jun 17 04:52 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XlmRoBertaForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [14]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = XlmRoBertaForQuestionAnswering.load(f"./{EXPORT_PATH}_spark_nlp_onnx") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

data = [
    ("What is the boiling point of water?", "Water boils at 100 degrees Celsius."),
    ("Who is the president of the USA?", "The president of the USA is Joe Biden."),
    ("What color is the sky?", "The sky is blue on a clear day."),
]
df = spark.createDataFrame(data, ["question", "context"])

result = pipeline.fit(df).transform(df)
result.select("answer.result").show(3, truncate=False)

+---------------------------+
|result                     |
+---------------------------+
|[What is the boiling point]|
|[is Joe Bi]                |
|[]                         |
+---------------------------+



That's it! You can now go wild and use hundreds of `RoBertaForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
